# Big Data Processing Coursework





In this short notebook, we will load and explore the movielens dataset. Specifically, this notebook covers:

Loading data in memory
Creating SQLContext
Creating Spark DataFrame
Group data by columns
Operating on columns
Running SQL Queries from a Spark DataFrame
Loading in a DataFrame

Build a recommendation system which uses transactional data linking a user and an item to get a list of items to recommend to the user. There are 2 approaches:
## Collaborative Filtering
In the MovieLens dataset, there are movies previously rated by a user, and the view is to attempt to identify if users who previously behaved similarly, ie liked/ disliked similar movies in the past, will have similar behaviors in the future. The input information is a user and the output is a list of items and their associated score. 
## Content based Filtering
In the MovieLens dataset, there are further information contained about each individual item, ie the movie and there are additional information supplied such as tags by users, genre information which can be used to further compare similar items.  The input information would be a model and the output information a list of items and their associated score.

Importing External files/Libraries¶

Ensure the environment variables for spark is setup correctly

In [9]:
import os
os.environ['SPARK_HOME']='/usr/lib/spark'

To use print function from python 3, use the from future command.  Ensure that jupyter notebook can find spark by using the findspark library, this references SPARK_HOME environment variable set up earlier.  Any external libraries imported need to be installed using pip install example, however if lacking admin permissions do pip install example --user  for example pip install sys --user.  Any error that has ImportError: No module named example means a pip install is required for a module named example.

In [10]:
#!/usr/bin/python
from __future__ import print_function 

import findspark
findspark.init()

from pyspark import SparkConf, SparkContext,sql

import sys
import re
import random
import array
import numpy as np
import scipy.sparse as sps


This program was tested with the following versions:

In [1]:
%reload_ext version_information
%version_information numpy, scipy, matplotlib, pyspark

Software versions
Python 2.7.5 64bit [GCC 4.8.5 20150623 (Red Hat 4.8.5-11)]
IPython 5.5.0
OS Linux 3.10.0 514.el7.x86_64 x86_64 with centos 7.3.1611 Core
numpy 1.7.1
scipy 0.12.1
matplotlib 2.0.2
Sun Jan 07 19:56:05 2018 GMT

## Load PySpark

In [2]:
sc = SparkContext(appName = "MovieLens").getOrCreate()
sqlContext = sql.SQLContext(sc)

## Custom Functions 

Functions for parsing movielens data and functions for comparing item similarity

In [3]:
sc.addPyFile("similarity.py")
sc.addPyFile("movielensfcn.py")
from movielensfcn import parseMovies, removeDuplicates, itemItem
from similarity import cosine_similarity, jaccard_similarity

First, let's get the data that we will working with in this notebook. We are using two files from the MovieLens dataset

In [1]:
#!wget --quiet http://www.grouplens.org/system/files/ml-100k.zip | unzip -q -o -d /data/movie-ratings/ | hadoop fs -put - /data/movie-ratings/

In [5]:
!hadoop fs -ls  '/data/movie-ratings'

Found 5 items
drwxr-xr-x   - alvarogr ECS640U          0 2017-12-11 11:52 /data/movie-ratings/cv
drwxr-xr-x   - hdfs     bigdata          0 2015-12-01 10:57 /data/movie-ratings/ml-10M100K
-rw-r--r--   3 hdfs     bigdata     522197 2015-12-01 10:17 /data/movie-ratings/movies.dat
-rw-r--r--   3 hdfs     bigdata  265105635 2015-12-01 10:17 /data/movie-ratings/ratings.dat
-rw-r--r--   3 hdfs     bigdata    3584119 2015-12-01 10:17 /data/movie-ratings/tags.dat


In [6]:
!hadoop fs -ls  '/data/movie-ratings/ml-10M100K'

Found 6 items
-rw-r--r--   3 hdfs bigdata      11135 2015-12-01 10:57 /data/movie-ratings/ml-10M100K/README.html
-rw-r--r--   3 hdfs bigdata        753 2015-12-01 10:57 /data/movie-ratings/ml-10M100K/allbut.pl
-rw-r--r--   3 hdfs bigdata     522197 2015-12-01 10:57 /data/movie-ratings/ml-10M100K/movies.dat
-rw-r--r--   3 hdfs bigdata  265105635 2015-12-01 10:57 /data/movie-ratings/ml-10M100K/ratings.dat
-rw-r--r--   3 hdfs bigdata       1092 2015-12-01 10:57 /data/movie-ratings/ml-10M100K/split_ratings.sh
-rw-r--r--   3 hdfs bigdata    3584119 2015-12-01 10:57 /data/movie-ratings/ml-10M100K/tags.dat


In [5]:
!hadoop fs -cat /data/movie-ratings/ml-10M100K/README.html

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">
  <head>
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
    <style type="text/css">
      h1 {
        color:#fc3;
        font-family:"Lucida Grande",Verdana,sans-serif; 
        font-size: 150%; 
        font-weight: normal; 
        margin:34px 0 0;
        background-color: #7A0019;
      }
      p {
        margin-left: 20px;
      }
      p.file_line_structure {
        margin-left: 40px;
      }
      table {
        margin-left: 30px;
      }
      th {
        text-align:left;
      }
    </style>

    <title>MovieLens 10M/100k Data Set README</title>
  </head>
  <body>
    <h1>
        Summary
    </h1>
    <p>
      This data set contains 10000054 ratings and 95580 tags 
      applied to 10681 movies by 71567 users of the 
      online movie recommender service <a href="ht

In [7]:
!hadoop fs -ls  '/data/movie-ratings/cv'

Found 2 items
drwxr-xr-x   - alvarogr ECS640U          0 2017-12-11 11:52 /data/movie-ratings/cv/10-item
drwxr-xr-x   - alvarogr ECS640U          0 2017-12-11 11:53 /data/movie-ratings/cv/5-fold


Review the contents of the movies.dat and ratings.dat files

In [8]:
ratings_file = "/data/movie-ratings/ratings.dat"
movies_file = "/data/movie-ratings/movies.dat"

In [9]:
ratings_raw = sc.textFile(ratings_file)
movies_raw = sc.textFile(movies_file)

Inspect the files to see what we are dealing with

In [12]:
ratings_raw.take(5)

[u'1::122::5::838985046',
 u'1::185::5::838983525',
 u'1::231::5::838983392',
 u'1::292::5::838983421',
 u'1::316::5::838983392']

## dat files
There is no header file
Notice that the columns are separated by :: 
Observe also that the field is in the following format:
user::movie::rating::timestamp

In [13]:
movies_raw.take(5)

[u'1::Toy Story (1995)::Adventure|Animation|Children|Comedy|Fantasy',
 u'2::Jumanji (1995)::Adventure|Children|Fantasy',
 u'3::Grumpier Old Men (1995)::Comedy|Romance',
 u'4::Waiting to Exhale (1995)::Comedy|Drama|Romance',
 u'5::Father of the Bride Part II (1995)::Comedy']

There is no header file
Notice that the columns are separated by :: Observe also that the field is in the following format
movie::titleandyear::genre

In [17]:
print('There are {0} rows in the {1}'.format(movies_raw.count(),movies_file))

There are 10681 rows in the /data/movie-ratings/movies.dat


In [19]:
print('There are {0} rows in the {1}'.format(ratings_raw.count(), ratings_file))

There are 10000054 rows in the /data/movie-ratings/ratings.dat


Since there are approximately 1M records, it may be faster to set the number of partitions on spark.  Since the movie file is relatively small with approximately 10K records we can hold in memory using collect

In [21]:
numPartitions =1000

In [ ]:
if (ratings_file.find('.dat')):
	movies= movies_raw.map(lambda line: re.split(r'::',line))\
        .map(lambda line: (int(line[0]),(line[1],line[2]))).collect()
	ratings = ratings_raw.map(lambda line: re.split(r'::',line))\
                        .map(lambda line: (int(line[0]),(int(line[1]),float(line[2]))))\
                        .partitionBy(numPartitions)
else:
	ratings_header = ratings_raw.take(1)[0]
	movies_header = movies_raw.take(1)[0]
	movies= movies_raw.filter(lambda line: line!=movies_header)\
                    .map(lambda line: re.split(r',',line)).map(lambda line: (int(line[1]),(line[0],line[2])))
	ratings = ratings_raw.filter(lambda line: line!=ratings_header)\
                    .map(lambda line: re.split(r',',line))\
                    .map(lambda x: (int(line[1]),(int(line[0]),float(line[2]))))\
                    .partitionBy(numPartitions)


In [ ]:
ratings.take(5)

In [ ]:
RatingsDF = ratings.toDF(['item_id','userid_rating'])

Let's check the type of RatingsDF

In [ ]:
type(RatingsDF)

The printSchema() method gives more details about the DataFrame’s schema and structure:

In [ ]:
RatingsDF.printSchema()

In [ ]:
RatingsDF.show(5)

How many movies do we have in the movies file?

In [ ]:
numMovies = ratings.values().map(lambda line: line[1]).distinct().count()
print("number of movies: {0}".format(numMovies))

How many users have rated our movies?

In [ ]:
numUsers = ratings.values().map(lambda line: line[0]).distinct().count()
print("number of users: {0}".format(numUsers))

In [ ]:
numRatings = ratings.count()
print("total number of ratings: {0}".format(numRatings))

Joining RDDs
Create RDDs for the same ratings and the movies files.

In [ ]:
user_ratings_data = ratings.join(ratings)

Join two dataframes and get only one 'item_id' column 

In [ ]:
user_ratingsDF = RatingsDF.join(RatingsDF,'item_id')

In [ ]:
user_ratings_data.take(5)

In [ ]:
user_ratingsDF.show(5)

In [ ]:
user_ratingsDF2 = sqlContext.createDataFrame(user_ratings_data,['item_id','userid_rating'])

In [ ]:
user_ratingsDF2.show(5)


Remove a rating if a user gives the same value for the same movie

In [ ]:
unique_joined_ratings = user_ratings_data.filter(removeDuplicates)

Map RDDs

In [ ]:
movie_pairs = unique_joined_ratings.map(itemItem).partitionBy(numPartitions)


Now group all ratings together for the same movie

In [ ]:
movie_pairs_ratings= movie_pairs.groupByKey()

In [ ]:
algorithms=["JACCARD","COSINE"]

In [ ]:
if algorithm == algorithms[0] :
	item_item_similarities = movie_pairs_ratings.mapValues(jaccard_similarity).persist()
elif algorithm == algorithms[1]  :
	item_item_similarities = movie_pairs_ratings.mapValues(cosine_similarity).persist()
else:
	item_item_similarities = movie_pairs_ratings.mapValues(cosine_similarity).persist()

In [ ]:
threshold = float(0.97)
topN= int(50)


In [ ]:
item_item_sorted=item_item_similarities.sortByKey()

In [ ]:
item_item_sorted.persist()

# Filter for movies with this sim that are "good" as defined by
# our quality thresholds above
filteredResults = item_item_sorted.filter(lambda((item_pair,similarity_occurence)): \
        (item_pair[0] == movie_id or item_pair[1] == movie_id) \
        and similarity_occurence[0] > threshold and similarity_occurence[1] > minOccurence)

if (topN==0):
    topN=10

results = filteredResults.map(lambda((x,y)): (y,x)).sortByKey(ascending = False)
resultsTopN = results.take(topN)
results.coalese(1).saveAsTextFile("movielens")




The join function combines two datasets (Key,ValueV) and (Key,ValueW) together to get (Key, (ValueV,ValueW)).  Let's join the movie and ratings file together to get meaningful recommendations

In [ ]:

   print "Top 10 similar movies for " + nameDict[movieID]
   for result in resultsTopN:
       (sim, pair) = result
#         Display the similarity result that isn't the movie we're looking at
       similarMovieID = pair[0]
       if (similarMovieID == movieID):
           similarMovieID = pair[1]
       print nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1])
	

In [ ]:
sc.stop()